In [ ]:
import requests
import json

In [ ]:
url = "https://bankaccountdata.gocardless.com/api/v2/token/new/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
with open('keys.json', 'rb') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
response = requests.post(url, headers=headers, data=json.dumps(data))

# Stampa il risultato
print(response.status_code)
print(response.json())  # mostra il token se la richiesta ha successo

In [ ]:
ACCESS_TOKEN = response.json()["access"]

In [ ]:
url = "https://bankaccountdata.gocardless.com/api/v2/institutions/"
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
}
params = {
    "country": "it"
}

response = requests.get(url, headers=headers, params=params)

# Stampa la risposta
print(response.status_code)
print(response.json())  # mostra la lista delle istituzioni

In [ ]:
institutions = response.json()

In [ ]:
for inst in institutions:
    print(inst['name'])

In [ ]:
my_bank = [ii for ii in institutions if ii['name'] == 'CREDEM Retail'][0]

In [ ]:
my_bank

In [ ]:
response = requests.post(
    url="https://bankaccountdata.gocardless.com/api/v2/requisitions/", 
    headers={
        "accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- Sostituisci con il token reale
    }, 
    json={
        "redirect": "http://www.yourwebpage.com",
        "institution_id": my_bank['id'],
        "reference": "111",
        "user_language": "IT"
    }
)

# Verifica la risposta
print(response.status_code)
print(response.json())


In [ ]:
requitision = response.json()

In [ ]:
response = requests.get(
    url="https://bankaccountdata.gocardless.com/api/v2/requisitions/",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }
)

data = response.json()

In [ ]:
requitision = data['results'][0]

In [ ]:
requitision

In [ ]:
response = requests.get(
    url=f"https://bankaccountdata.gocardless.com/api/v2/requisitions/{requitision['id']}/",
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
    }
)
# Verifica la risposta
print(response.status_code)
print(response.json())


In [ ]:
user_profile = response.json()

In [ ]:
response = requests.get(
    url=f"https://bankaccountdata.gocardless.com/api/v2/accounts/{user_profile['accounts'][0]}/transactions/" ,
    headers={
        "accept": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"  # <-- sostituisci con il token reale
    }
)
# Verifica la risposta
print(response.status_code)
print(response.json())

In [ ]:
transactions = response.json()

In [ ]:
transactions["transactions"]["booked"][45]

In [ ]:
for ii in transactions["transactions"]["booked"]:
    print(ii['internalTransactionId'], " --- ", ii['transactionId'])

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
from data_models.transaction import ApiTransactionRaw, ConsolidatedTransaction

In [ ]:
raw_dict = transactions["transactions"]["booked"][45]
raw_dict

In [ ]:
raw_dict = {'transactionId': '001-010-010-00000-000848324-00000$900000090-20250619-0005020-001',
 'bookingDate': '2025-06-19',
 'valueDate': '2025-06-17',
 'transactionAmount': {'amount': '-6.00', 'currency': 'EUR'},
 'remittanceInformationUnstructured': 'PAGOBANCOMAT 17/06/25,ORE 22:02,TESS.802574245 GOUST DI VALOCATERIN..                                                                                 ',
 'internalTransactionId': '9de881431c7fec6de689ffdd0b4e66fd'}

In [ ]:
tx = ApiTransactionRaw(
    booking_dt=raw_dict['bookingDate'],
    value_dt=raw_dict['valueDate'],
    amount=raw_dict['transactionAmount']['amount'],
    description=raw_dict['remittanceInformationUnstructured'],
    external_id=raw_dict['transactionId'],
)

In [ ]:
tx

In [ ]:
cons_tx = ConsolidatedTransaction.model_validate(tx.model_dump())

In [ ]:
cons_tx.hash_key

In [ ]:
ConsolidatedTransaction.from_raw(tx)